https://towardsdatascience.com/predict-the-number-of-likes-on-instagram-a7ec5c020203

# Imports

In [216]:
import pandas as pd
import json
import numpy as np
import re
import pickle
from datetime import datetime

from utils import clean_text, clean_sentence

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Data Loading

In [291]:
train = pd.read_csv('data_like/customer_train.csv')
test = pd.read_csv('data_like/customer_test.csv')
print(train.shape)
print(test.shape)

(50000, 15)
(54245, 15)


In [3]:
stories = pd.read_csv('data_like/stories_description.csv')
print(stories.shape)

(959, 2)


In [4]:
train_y = pd.read_csv('data_like/stories_reaction_train.csv')
test_y = pd.read_csv('data_like/stories_reaction_test.csv')
print(train_y.shape)
print(test_y.shape)

(473141, 4)
(172049, 4)


In [5]:
transaction = pd.read_csv('data_like/transactions.csv')
print(transaction.shape)

(3951115, 6)


In [116]:
description = pd.read_excel('data_like/Hack_desc.xlsx')
description.index = description['customer']
description = description.drop(labels=['customer'], axis=1)
description

,Unnamed: 1
customer,
customer_id,Идентификатор клиента
product_X,"Статус продукта. OPN - открыт, но не утилизиро..."
gender_cd,Пол. M - мужской. F - женский
age,Возраст в годах
marital_status_cd,Семейный статус. См. словарь соответствия
children_cnt,Количество детей в штуках
first_session_dttm,Дата и время первой сессии в приложении или ли...
job_position_cd,Категория занимаемой должности. См. словарь со...
job_title,Занимаемая должность


In [122]:
description['Unnamed: 1'].loc['event']

'Тип действия. like - лайк или сохранение. view - глубокий просмотр (более 10 секунд). skip - пролистанная история (менее 5 секунд). dislike - дизлайк'

# Extracting text features from jsons

In [30]:
REG_TEXT = "text\":\"(.*?)\","
REG_NAME = "name\":\"(.*?)\","

r = re.findall(REG_NAME, stories['story_json'][10])

In [31]:
decoded_stories = dict()
for i in stories.index:
    cur_story = stories['story_json'][i]
    story_info = dict()
    name = re.findall(REG_NAME, cur_story)
    story_info['name'] = "" if len(name) == 0 else clean_sentence(name[0])
    story_info['text'], story_info['swipes'], story_info['n_words'] = clean_text(re.findall(REG_TEXT, cur_story))
    decoded_stories[stories['story_id'][i]] = story_info

In [32]:
# with open("stories", "wb") as fp:
#     pickle.dump(decoded_stories, fp)

In [33]:
# with open("stories", "rb") as fp:
#     b = pickle.load(fp)

# Data Analysis 

## 1) Likes, Dislikes and other stuff

In [324]:
len(list(set(train_y.customer_id).intersection(set(test_y.customer_id))))

14260

In [326]:
len(list(set(train_y.story_id).intersection(set(test_y.story_id))))

670

#### Some customers from train are also in test as well as stories

In [342]:
inter = list(set(train_y.customer_id).intersection(set(test_y.customer_id)))
num = 4

In [345]:
train_y[train_y.customer_id==inter[num]]

,customer_id,story_id,event_dttm,event
444336,917520,129,2018-05-31 05:05:34,view
444337,917520,129,2018-06-01 13:14:11,view
444338,917520,133,2018-05-28 11:58:47,skip
444339,917520,136,2018-05-28 11:58:47,view
444340,917520,138,2018-06-20 15:50:13,skip
444341,917520,171,2018-05-24 04:45:37,view
444342,917520,202,2018-06-14 01:43:54,view
444343,917520,202,2018-06-14 14:50:16,skip
444344,917520,202,2018-06-15 14:01:51,skip
444345,917520,216,2018-06-27 06:21:37,view


#### Customers can watch the same stories few times, can be a good feature

In [346]:
test_y[test_y.customer_id==inter[num]]

,customer_id,story_id,event_dttm,answer_id
10145,917520,1415,2018-08-02 05:05:02,10145
10146,917520,1381,2018-08-02 05:05:02,10146
10147,917520,1367,2018-08-02 05:05:02,10147
10148,917520,1246,2018-08-02 05:05:02,10148
10149,917520,1012,2018-08-02 05:05:02,10149
41767,917520,1439,2018-08-06 03:20:47,41767
41768,917520,228,2018-08-06 03:20:47,41768
41769,917520,1433,2018-08-06 03:20:47,41769
41770,917520,1418,2018-08-06 03:20:47,41770
41771,917520,1220,2018-08-06 03:20:47,41771


In [344]:
test_y.head()

,customer_id,story_id,event_dttm,answer_id
0,466906,1152,2018-08-01 00:00:46,0
1,466906,1355,2018-08-01 00:00:46,1
2,146395,537,2018-08-01 00:02:08,2
3,897303,915,2018-08-01 00:02:30,3
4,50094,1415,2018-08-01 00:03:13,4


In [38]:
train_y.event.value_counts() / len(train_y)

view       0.512135
skip       0.354159
like       0.119810
dislike    0.013896
Name: event, dtype: float64

In [127]:
# train_y.loc[train_y.event == 'dislike'].story_id.value_counts().iloc[0:50]

In [128]:
# train_y.loc[train_y.event == 'like'].story_id.value_counts()[0: 50]

top_liked : top_disliked  ==  31 <br>
top_viewed : top_disliked  ==  22 <br>
top_skiped : top_disliked  ==  20 <br>
top_viewed : top_liked  == 29 <br>
top_skiped : top_liked == 24 <br>
top_viewed : top_skiped == 27 <br>

### ???? disliked are much more similar to liked, may be it is a misclick of user ?????

## 2) Users info

In [305]:
# train.head()

In [293]:
nan = 'NAN'

In [294]:
# train = train.fillna(nan)

In [295]:
# Delete rare jobs because it may be useless during one-hot encoding
train.job_title = [job.lower() if isinstance(job, str) else job for job in train.job_title]
common_jobs = set(train['job_title'].value_counts().keys()[train['job_title'].value_counts() > 50])
train.job_title = [job if job in common_jobs else nan for job in train.job_title]

In [296]:
common_n_children = set(train['children_cnt'].value_counts().keys()[train['children_cnt'].value_counts() > 19])
train.children_cnt = [ch if ch in common_n_children else nan for ch in train.children_cnt]

In [297]:
common_n_status = set(train['marital_status_cd'].value_counts().keys()[train['marital_status_cd'].value_counts() > 2])
train.marital_status_cd = [ch if ch in common_n_status else nan for ch in train.marital_status_cd]

#### Let's watch by eyes to data

In [306]:
# categorial_feat = ['product_0', 'product_1', 'product_2', 'product_3', 
#                    'product_4', 'product_5', 'product_6', 'gender_cd',
#                    'marital_status_cd', 'children_cnt', 'job_position_cd', 'job_title']
# for col in categorial_feat:
#     print('FEATURE:', col)
#     print(train[col].value_counts())
#     print('Nans:', len(train) - np.sum(train[col].value_counts()))
#     print('_________')

#### Let's see data of registration

In [310]:
years = []
for i in range(len(train)):
    if isinstance(train.first_session_dttm[i], str):
        years.append(datetime.strptime(train.first_session_dttm[i], '%Y-%m-%d %H:%M:%S').year)
    else:
        years.append('NaN')

In [311]:
np.unique(years)

array(['2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019',
       'NaN'], dtype='<U21')

#### Let's see age

In [302]:
train.age = train.age.fillna(0)
np.unique(train.age)

array([ 0., 15., 20., 25., 30., 35., 40., 45., 50., 55., 60., 65., 70.,
       75., 80., 85.])

### can use it as a categorial feature

## 3) Transactions

In [352]:
description.loc[transaction.columns].iloc[3: ]

,Unnamed: 1
customer,
customer_id,Идентификатор клиента
transaction_month,Месяц транзакции
transaction_day,День транзакции
transaction_amt,Сумма транзакции в рублях
merchant_id,"Идентификатор магазина, в котором была соверше..."
merchant_mcc,MCC-категория магазина


In [348]:
transaction

,customer_id,transaction_month,transaction_day,transaction_amt,merchant_id,merchant_mcc
0,855115,7,3,1500,4554547,5411
1,997036,6,6,0,1657528,5411
2,398237,5,24,2500,26375569,5813
3,997036,6,2,0,16304402,5411
4,291636,7,25,0,1259505,5411
5,844503,5,24,3500,12790541,5411
6,844503,6,22,3500,12790541,5411
7,862192,7,14,0,65419694,5814
8,862192,5,26,0,97570081,5411
9,862192,7,11,0,65419694,5814


In [357]:
print(len(transaction.merchant_id.value_counts()))
transaction.merchant_id.value_counts()

664198


77187702     85735
102251864    20277
85172426     18291
83111364     14511
52565850     14124
92715864     13939
2480383       9724
99457078      9657
120710589     8242
95677293      6399
39099863      6104
85380022      5753
65824772      5646
97340721      4689
93571120      4571
77753195      4509
99419203      4264
77266884      3911
74322178      3659
111484690     3499
105645284     3494
25686550      3369
94600451      3213
85176168      2788
72282305      2634
117817718     2531
119257786     2424
11199786      2403
59562756      2350
40218735      2304
             ...  
28573911         1
126726791        1
93131411         1
112509135        1
45562032         1
787121           1
116906156        1
114055866        1
64401895         1
1483237          1
117145895        1
14111195         1
22956280         1
13670690         1
101876065        1
112847210        1
26405228         1
122825076        1
3715393          1
60129665         1
18200966         1
18215311    

#### There are a lot of popular shops, maybe like x5 

In [356]:
len(transaction.merchant_mcc.value_counts())

409

In [358]:
transaction.customer_id.value_counts()

514522     804
588639     788
333041     768
588106     700
673392     690
878241     674
533341     666
67304      653
334074     634
110621     623
141608     621
177704     608
335239     602
744399     598
540513     596
43361      588
598655     578
141084     564
634747     562
192793     559
130931     559
87429      557
249174     552
49203      546
301112     540
153166     539
135465     539
835803     536
8481       536
456478     534
          ... 
34462        1
775470       1
667131       1
675777       1
969138       1
342357       1
695788       1
884801       1
668720       1
540920       1
439181       1
741349       1
133167       1
828986       1
974677       1
126204       1
559694       1
727308       1
784656       1
957106       1
185147       1
462450       1
488335       1
570468       1
739851       1
542958       1
462666       1
743422       1
1010076      1
195566       1
Name: customer_id, Length: 46948, dtype: int64

#### Ideas: <br> 1. average amount of money for transaction for each user, two features: average over all transactions and of >0 rubles transactions<br> 2. average number of transaction per month <br> 3. Maybe if user visits top popular shops 

In [359]:
transaction[transaction.customer_id == 564201]

,customer_id,transaction_month,transaction_day,transaction_amt,merchant_id,merchant_mcc
11,564201,7,5,0,6646839,5999
12,564201,7,3,0,8215883,5411
15,564201,7,24,0,18567776,5912
433,564201,5,27,500,16324064,5812
126658,564201,7,19,0,1495825,5331
126662,564201,6,26,0,1495825,5331
126663,564201,6,4,0,1495825,5331
126665,564201,7,5,0,1933370,5331
126668,564201,6,5,0,1495825,5331
126670,564201,6,23,0,1495825,5331
